## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Dec. 8th Dec. 15th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,December-11-2024,Condo/Co-op,1402 Captains Way,Jupiter,FL,33477.0,1895000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1402-Captain...,Beaches MLS,RX-11030902,N,Y,26.909366,-80.086902


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
12,PAST SALE,2024-12-16,Condo/Co-op,240 Crestwood Cir #104,Royal Palm Beach,FL,33411.0,295000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Royal-Palm-Beach/240...,Beaches MLS,RX-11017718,N,Y,26.691940,-80.245540
70,PAST SALE,2024-12-13,Condo/Co-op,555 Saxony L,Delray Beach,FL,33446.0,72500.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/555-Sax...,Beaches MLS,RX-10952386,N,Y,26.440263,-80.150386
32,PAST SALE,2024-12-13,Condo/Co-op,307 Pine Ridge Cir Unit C-1,Green Acres,FL,33463.0,159500.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Greenacres/307-Pine-...,MARMLS,A11663911,N,Y,26.627008,-80.132008
38,PAST SALE,2024-12-13,Condo/Co-op,3425 San Bernadino Dr Unit C,Delray Beach,FL,33445.0,328000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/3425-Sa...,Beaches MLS,F10461037,N,Y,26.438799,-80.111516
39,PAST SALE,2024-12-13,Condo/Co-op,13777 Via Aurora Unit A,Delray Beach,FL,33484.0,175000.0,2.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/13777-V...,Beaches MLS,RX-11013819,N,Y,26.472417,-80.137173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,PAST SALE,2024-12-09,Condo/Co-op,1055 Crystal Way Unit D,Delray Beach,FL,33444.0,268000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1055-Cr...,Beaches MLS,F10464436,N,Y,26.432332,-80.084480
33,PAST SALE,2024-12-09,Condo/Co-op,4115 San Marino Blvd #304,West Palm Beach,FL,33409.0,188000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/4115...,Beaches MLS,RX-11021629,N,Y,26.737891,-80.106618
46,PAST SALE,2024-12-09,Condo/Co-op,353 S Us Highway 1 Unit D102,Jupiter,FL,33477.0,620000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/353-S-US-Hig...,Beaches MLS,RX-11001551,N,Y,26.929360,-80.077743
51,PAST SALE,2024-12-09,Condo/Co-op,1038 Cornwall B,Boca Raton,FL,33434.0,145000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1038-Corn...,Beaches MLS,RX-10998207,N,Y,26.386660,-80.180173


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

70


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$402,996


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$329


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$28,209,690


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
112
Input Previous Week Condo Average Sales Price:
434716
Input Previous Week Condo Average PSF:
297
Input Previous Week Condo Sales total (ex: 198_000_000)
48_700_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [24]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [25]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [26]:
pd.set_option('display.max_columns',None)

In [27]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-13,Condo/Co-op,455 Worth Ave #407,Palm Beach,FL,33480.0,2200000.0,2.0,2.0,Four 55 Worth Ave Corp,955.0,24419.0,1971.0,NaN,2304.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Beaches MLS,RX-10998885,N,Y,26.701559,-80.043327,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Christine Gibbons,"Sotheby's Intl. Realty, Inc.",None,None,Non Agent,Non-Member Selling Office,None,None,1,orange


In [28]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [29]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-13,Condo/Co-op,455 Worth Ave #407,Palm Beach,FL,33480.0,2200000.0,2.0,2.0,Four 55 Worth Ave Corp,955.0,24419.0,1971.0,NaN,2304.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Beaches MLS,RX-10998885,N,Y,26.701559,-80.043327,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Christine Gibbons,"Sotheby's Intl. Realty, Inc.",None,None,Non Agent,Non-Member Selling Office,None,None,1,orange
1,PAST SALE,2024-12-12,Condo/Co-op,2660 S Ocean Blvd Unit 205s,Palm Beach,FL,33480.0,2100000.0,2.0,2.5,Beach Point Condo,1900.0,NaN,1979.0,NaN,1105.0,2308.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-10955832,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Wendy Bowes,The Corcoran Group,William McManus,The Corcoran Group,Scott Gordon,Douglas Elliman,None,None,2,blue
2,PAST SALE,2024-12-11,Condo/Co-op,1402 Captains Way,Jupiter,FL,33477.0,1895000.0,2.0,2.0,Admirals Cove,1952.0,NaN,1987.0,NaN,971.0,2358.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1402-Captain...,Beaches MLS,RX-11030902,N,Y,26.909366,-80.086902,https://www.redfin.com/FL/Jupiter/1402-Captain...,Stephen Rockoff,The Corcoran Group,None,None,Donna Hutchins,Douglas Elliman (Jupiter),None,None,3,blue
3,PAST SALE,2024-12-13,Condo/Co-op,3912 S Ocean Blvd #1115,Highland Beach,FL,33487.0,1210000.0,2.0,2.0,Regency Highland,1521.0,NaN,1976.0,NaN,796.0,1171.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-11008378,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,Peter Bauch,HomeSmart,None,None,4,blue
4,PAST SALE,2024-12-12,Condo/Co-op,701 S Olive Ave #1004,West Palm Beach,FL,33401.0,1025000.0,2.0,2.5,Two City Plaza,1625.0,365.0,2008.0,NaN,631.0,1562.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-10972690,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Colleen Foley,Luxe Life Realty LLC,None,None,Joseph Columbo,Douglas Elliman,None,None,5,blue
5,PAST SALE,2024-12-12,Condo/Co-op,127 Tresana Blvd #63,Jupiter,FL,33478.0,875000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2015.0,NaN,412.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/127-Tresana-...,Beaches MLS,RX-11031637,N,Y,26.926730,-80.156887,https://www.redfin.com/FL/Jupiter/127-Tresana-...,Sylvia Isaacs,Echo Fine Properties,Jeffrey Lichtenstein,Echo Fine Properties,Marisol Peralta,Robert Slack LLC,None,None,6,blue
6,PAST SALE,2024-12-09,Condo/Co-op,4740 S Ocean Blvd #503,Highland Beach,FL,33487.0,800000.0,2.0,2.0,Braemar Isle Condo,2050.0,NaN,1982.0,NaN,390.0,2039.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11023545,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Mary Ann Phillips,RE/MAX Select Group,None,None,Mary Ann Phillips,RE/MAX Select Group,None,None,7,blue
7,PAST SALE,2024-12-10,Condo/Co-op,1748 Jupiter Cove Dr #220,Jupiter,FL,33469.0,775000.0,2.0,2.0,Jupiter Cove Condo,1300.0,NaN,1985.0,NaN,596.0,1441.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1748-Jupiter...,Beaches MLS,RX-10984654,N,Y,26.948690,-80.085844,https://www.redfin.com/FL/Jupiter/1748-Jupiter...,Carol Hartmanis PA,"NV Realty Group, LLC",None,None,Jacqueline Hanna,William Raveis Palm Beach LLC,None,None,8,blue
8,PAST SALE,2024-12-11,Condo/Co-op,108 Palm Bay Dr Unit A,Palm Beach Gardens,FL,33418.0,765000.0,2.0,2.0,Palm Bay Club Condo,2195.0,NaN,2002.0

In [30]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Palm Beach County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [31]:
m.save('index.html')

## Data snagger

In [32]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-13,Condo/Co-op,455 Worth Ave #407,Palm Beach,FL,33480.0,2200000.0,2.0,2.0,Four 55 Worth Ave Corp,955.0,24419.0,1971.0,NaN,2304.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Beaches MLS,RX-10998885,N,Y,26.701559,-80.043327,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Christine Gibbons,"Sotheby's Intl. Realty, Inc.",None,None,Non Agent,Non-Member Selling Office,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [34]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_12162024


# CREATE TEMPLATE 

In [35]:
muni_set = set(df_top_ten['CITY'])

In [36]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [37]:
df_top_ten.reset_index(inplace=True,drop=True)

In [38]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [39]:
top_sale

'Four 55 Worth Ave Corp at 455 Worth Ave #407 in Palm Beach'

In [40]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [41]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [42]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-13,Condo/Co-op,455 Worth Ave #407,Palm Beach,FL,33480.0,2200000.0,2.0,2.0,Four 55 Worth Ave Corp,955.0,24419.0,1971.0,NaN,2304.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Beaches MLS,RX-10998885,N,Y,26.701559,-80.043327,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Christine Gibbons,"Sotheby's Intl. Realty, Inc.",None,None,Non Agent,Non-Member Selling Office,None,None,1,orange,Four 55 Worth Ave Corp at 455 Worth Ave #407 i...
1,PAST SALE,2024-12-12,Condo/Co-op,2660 S Ocean Blvd Unit 205s,Palm Beach,FL,33480.0,2100000.0,2.0,2.5,Beach Point Condo,1900.0,NaN,1979.0,NaN,1105.0,2308.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-10955832,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Wendy Bowes,The Corcoran Group,William McManus,The Corcoran Group,Scott Gordon,Douglas Elliman,None,None,2,blue,Beach Point Condo at 2660 S Ocean Blvd Unit 20...
2,PAST SALE,2024-12-11,Condo/Co-op,1402 Captains Way,Jupiter,FL,33477.0,1895000.0,2.0,2.0,Admirals Cove,1952.0,NaN,1987.0,NaN,971.0,2358.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1402-Captain...,Beaches MLS,RX-11030902,N,Y,26.909366,-80.086902,https://www.redfin.com/FL/Jupiter/1402-Captain...,Stephen Rockoff,The Corcoran Group,None,None,Donna Hutchins,Douglas Elliman (Jupiter),None,None,3,blue,Admirals Cove at 1402 Captains Way in Jupiter
3,PAST SALE,2024-12-13,Condo/Co-op,3912 S Ocean Blvd #1115,Highland Beach,FL,33487.0,1210000.0,2.0,2.0,Regency Highland,1521.0,NaN,1976.0,NaN,796.0,1171.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-11008378,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,Peter Bauch,HomeSmart,None,None,4,blue,Regency Highland at 3912 S Ocean Blvd #1115 in...
4,PAST SALE,2024-12-12,Condo/Co-op,701 S Olive Ave #1004,West Palm Beach,FL,33401.0,1025000.0,2.0,2.5,Two City Plaza,1625.0,365.0,2008.0,NaN,631.0,1562.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-10972690,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Colleen Foley,Luxe Life Realty LLC,None,None,Joseph Columbo,Douglas Elliman,None,None,5,blue,Two City Plaza at 701 S Olive Ave #1004 in Wes...
5,PAST SALE,2024-12-12,Condo/Co-op,127 Tresana Blvd #63,Jupiter,FL,33478.0,875000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2015.0,NaN,412.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/127-Tresana-...,Beaches MLS,RX-11031637,N,Y,26.926730,-80.156887,https://www.redfin.com/FL/Jupiter/127-Tresana-...,Sylvia Isaacs,Echo Fine Properties,Jeffrey Lichtenstein,Echo Fine Properties,Marisol Peralta,Robert Slack LLC,None,None,6,blue,Jupiter Country Club at 127 Tresana Blvd #63 i...
6,PAST SALE,2024-12-09,Condo/Co-op,4740 S Ocean Blvd #503,Highland Beach,FL,33487.0,800000.0,2.0,2.0,Braemar Isle Condo,2050.0,NaN,1982.0,NaN,390.0,2039.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11023545,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Mary Ann Phillips,RE/MAX Select Group,None,None,Mary Ann Phillips,RE/MAX Select Group,None,None,7,blue,Braemar Isle Condo at 4740 S Ocean Blvd #503 i...
7,PAST SALE,2024-12-10,Condo/Co-op,1748 Jupiter Cove Dr #220,Jupiter,FL,33469.0,775000.0,2.0,2.0,Jupiter Cove Condo,1300.0,NaN,1985.0,NaN,596.0,1441.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1748-Jupiter...,B

In [43]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Four 55 Worth Ave Corp closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $745,000M to $2,200,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Palm Beach Gardens, Highland Beach, Jupiter, Palm Beach, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 70 condo sales totaling $28,209,690 million from Dec. 8th to Dec. 15th. The previous week, brokers closed 112 condo sales totaling $48,700,000.

Last week’s units sold for an average of $402,996, lower

In [44]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [45]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-13,Condo/Co-op,455 Worth Ave #407,Palm Beach,FL,33480.0,2200000.0,2.0,2.0,Four 55 Worth Ave Corp,955.0,24419.0,1971.0,NaN,2304.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Beaches MLS,RX-10998885,N,Y,26.701559,-80.043327,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Christine Gibbons,"Sotheby's Intl. Realty, Inc.",None,None,Non Agent,Non-Member Selling Office,None,None,1,orange,Four 55 Worth Ave Corp at 455 Worth Ave #407 i...
1,PAST SALE,2024-12-12,Condo/Co-op,2660 S Ocean Blvd Unit 205s,Palm Beach,FL,33480.0,2100000.0,2.0,2.5,Beach Point Condo,1900.0,NaN,1979.0,NaN,1105.0,2308.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-10955832,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Wendy Bowes,The Corcoran Group,William McManus,The Corcoran Group,Scott Gordon,Douglas Elliman,None,None,2,blue,Beach Point Condo at 2660 S Ocean Blvd Unit 20...
2,PAST SALE,2024-12-11,Condo/Co-op,1402 Captains Way,Jupiter,FL,33477.0,1895000.0,2.0,2.0,Admirals Cove,1952.0,NaN,1987.0,NaN,971.0,2358.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1402-Captain...,Beaches MLS,RX-11030902,N,Y,26.909366,-80.086902,https://www.redfin.com/FL/Jupiter/1402-Captain...,Stephen Rockoff,The Corcoran Group,None,None,Donna Hutchins,Douglas Elliman (Jupiter),None,None,3,blue,Admirals Cove at 1402 Captains Way in Jupiter
3,PAST SALE,2024-12-13,Condo/Co-op,3912 S Ocean Blvd #1115,Highland Beach,FL,33487.0,1210000.0,2.0,2.0,Regency Highland,1521.0,NaN,1976.0,NaN,796.0,1171.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-11008378,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,Peter Bauch,HomeSmart,None,None,4,blue,Regency Highland at 3912 S Ocean Blvd #1115 in...
4,PAST SALE,2024-12-12,Condo/Co-op,701 S Olive Ave #1004,West Palm Beach,FL,33401.0,1025000.0,2.0,2.5,Two City Plaza,1625.0,365.0,2008.0,NaN,631.0,1562.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-10972690,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Colleen Foley,Luxe Life Realty LLC,None,None,Joseph Columbo,Douglas Elliman,None,None,5,blue,Two City Plaza at 701 S Olive Ave #1004 in Wes...
5,PAST SALE,2024-12-12,Condo/Co-op,127 Tresana Blvd #63,Jupiter,FL,33478.0,875000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2015.0,NaN,412.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/127-Tresana-...,Beaches MLS,RX-11031637,N,Y,26.926730,-80.156887,https://www.redfin.com/FL/Jupiter/127-Tresana-...,Sylvia Isaacs,Echo Fine Properties,Jeffrey Lichtenstein,Echo Fine Properties,Marisol Peralta,Robert Slack LLC,None,None,6,blue,Jupiter Country Club at 127 Tresana Blvd #63 i...
6,PAST SALE,2024-12-09,Condo/Co-op,4740 S Ocean Blvd #503,Highland Beach,FL,33487.0,800000.0,2.0,2.0,Braemar Isle Condo,2050.0,NaN,1982.0,NaN,390.0,2039.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11023545,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Mary Ann Phillips,RE/MAX Select Group,None,None,Mary Ann Phillips,RE/MAX Select Group,None,None,7,blue,Braemar Isle Condo at 4740 S Ocean Blvd #503 i...
7,PAST SALE,2024-12-10,Condo/Co-op,1748 Jupiter Cove Dr #220,Jupiter,FL,33469.0,775000.0,2.0,2.0,Jupiter Cove Condo,1300.0,NaN,1985.0,NaN,596.0,1441.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1748-Jupiter...,B

In [48]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Four 55 Worth Ave Corp closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $745,000M to $2,200,000M
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Palm Beach Gardens, Highland Beach, Jupiter, Palm Beach, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 70 condo sales totaling $28,209,690 million from Dec. 8th to Dec. 15th. The previous week, brokers closed 112 condo sales totaling $48,700,000.

Last week’s units sold for an average of $402,996, lower

In [49]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Palm-Beach/455-Worth-Ave-33480/unit-407/home/42433633


In [50]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/2660-S-Ocean-Blvd-33480/unit-205S/home/42642136


In [51]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Jupiter/100-Ocean-Trail-Way-33477/unit-1305/home/42401279


In [52]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Palm-Beach-Gardens/108-Palm-Bay-Dr-33418/unit-A/home/42571386


In [53]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/455-Worth-Ave-33480/unit-407/home/42433633


In [54]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Palm-Beach-Gardens/108-Palm-Bay-Dr-33418/unit-A/home/42571386


In [55]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-13,Condo/Co-op,455 Worth Ave #407,Palm Beach,FL,33480.0,2200000.0,2.0,2.0,Four 55 Worth Ave Corp,955.0,24419.0,1971.0,NaN,2304.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Beaches MLS,RX-10998885,N,Y,26.701559,-80.043327,https://www.redfin.com/FL/Palm-Beach/455-Worth...,Christine Gibbons,"Sotheby's Intl. Realty, Inc.",None,None,Non Agent,Non-Member Selling Office,None,None,1,orange,Four 55 Worth Ave Corp at 455 Worth Ave #407 i...
1,PAST SALE,2024-12-12,Condo/Co-op,2660 S Ocean Blvd Unit 205s,Palm Beach,FL,33480.0,2100000.0,2.0,2.5,Beach Point Condo,1900.0,NaN,1979.0,NaN,1105.0,2308.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Beaches MLS,RX-10955832,N,Y,26.623992,-80.037641,https://www.redfin.com/FL/Palm-Beach/2660-S-Oc...,Wendy Bowes,The Corcoran Group,William McManus,The Corcoran Group,Scott Gordon,Douglas Elliman,None,None,2,blue,Beach Point Condo at 2660 S Ocean Blvd Unit 20...
2,PAST SALE,2024-12-11,Condo/Co-op,1402 Captains Way,Jupiter,FL,33477.0,1895000.0,2.0,2.0,Admirals Cove,1952.0,NaN,1987.0,NaN,971.0,2358.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1402-Captain...,Beaches MLS,RX-11030902,N,Y,26.909366,-80.086902,https://www.redfin.com/FL/Jupiter/1402-Captain...,Stephen Rockoff,The Corcoran Group,None,None,Donna Hutchins,Douglas Elliman (Jupiter),None,None,3,blue,Admirals Cove at 1402 Captains Way in Jupiter
3,PAST SALE,2024-12-13,Condo/Co-op,3912 S Ocean Blvd #1115,Highland Beach,FL,33487.0,1210000.0,2.0,2.0,Regency Highland,1521.0,NaN,1976.0,NaN,796.0,1171.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-11008378,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,Peter Bauch,HomeSmart,None,None,4,blue,Regency Highland at 3912 S Ocean Blvd #1115 in...
4,PAST SALE,2024-12-12,Condo/Co-op,701 S Olive Ave #1004,West Palm Beach,FL,33401.0,1025000.0,2.0,2.5,Two City Plaza,1625.0,365.0,2008.0,NaN,631.0,1562.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-10972690,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Colleen Foley,Luxe Life Realty LLC,None,None,Joseph Columbo,Douglas Elliman,None,None,5,blue,Two City Plaza at 701 S Olive Ave #1004 in Wes...
5,PAST SALE,2024-12-12,Condo/Co-op,127 Tresana Blvd #63,Jupiter,FL,33478.0,875000.0,2.0,2.0,Jupiter Country Club,2124.0,NaN,2015.0,NaN,412.0,1293.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/127-Tresana-...,Beaches MLS,RX-11031637,N,Y,26.926730,-80.156887,https://www.redfin.com/FL/Jupiter/127-Tresana-...,Sylvia Isaacs,Echo Fine Properties,Jeffrey Lichtenstein,Echo Fine Properties,Marisol Peralta,Robert Slack LLC,None,None,6,blue,Jupiter Country Club at 127 Tresana Blvd #63 i...
6,PAST SALE,2024-12-09,Condo/Co-op,4740 S Ocean Blvd #503,Highland Beach,FL,33487.0,800000.0,2.0,2.0,Braemar Isle Condo,2050.0,NaN,1982.0,NaN,390.0,2039.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11023545,N,Y,26.393421,-80.069121,https://www.redfin.com/FL/Highland-Beach/4740-...,Mary Ann Phillips,RE/MAX Select Group,None,None,Mary Ann Phillips,RE/MAX Select Group,None,None,7,blue,Braemar Isle Condo at 4740 S Ocean Blvd #503 i...
7,PAST SALE,2024-12-10,Condo/Co-op,1748 Jupiter Cove Dr #220,Jupiter,FL,33469.0,775000.0,2.0,2.0,Jupiter Cove Condo,1300.0,NaN,1985.0,NaN,596.0,1441.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/1748-Jupiter...,B

## Time on Market Calculator

In [57]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 2, 1) ## List (Earlier) date
date2 = datetime(2024, 12, 12) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

315


## Clean CSV for Datawrapper Chart

In [58]:
chart_df = df_top_ten

In [59]:
chart_df = chart_df.fillna(" ")

In [60]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [61]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [62]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [63]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [64]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [65]:
csv_date_string = today.strftime("%m_%d_%Y")

In [66]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [67]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-12-13,455 Worth Ave #407 in Palm Beach,"$2,200,000",2,2.0,955,1971,"$2,304",Beaches MLS,RX-10998885,"Christine Gibbons Sotheby's Intl. Realty, Inc....",Non Agent Non-Member Selling Office
1,2024-12-12,2660 S Ocean Blvd Unit 205s in Palm Beach,"$2,100,000",2,2.5,"1,900",1979,"$1,105",Beaches MLS,RX-10955832,Wendy Bowes The Corcoran Group William McManus...,Scott Gordon Douglas Elliman
2,2024-12-11,1402 Captains Way in Jupiter,"$1,895,000",2,2.0,"1,952",1987,$971,Beaches MLS,RX-11030902,Stephen Rockoff The Corcoran Group,Donna Hutchins Douglas Elliman (Jupiter)
3,2024-12-13,3912 S Ocean Blvd #1115 in Highland Beach,"$1,210,000",2,2.0,"1,521",1976,$796,Beaches MLS,RX-11008378,Matthew Bachrad William Raveis Real Estate Dan...,Peter Bauch HomeSmart
4,2024-12-12,701 S Olive Ave #1004 in West Palm Beach,"$1,025,000",2,2.5,"1,625",2008,$631,Beaches MLS,RX-10972690,Colleen Foley Luxe Life Realty LLC,Joseph Columbo Douglas Elliman
5,2024-12-12,127 Tresana Blvd #63 in Jupiter,"$875,000",2,2.0,"2,124",2015,$412,Beaches MLS,RX-11031637,Sylvia Isaacs Echo Fine Properties Jeffrey Lic...,Marisol Peralta Robert Slack LLC
6,2024-12-09,4740 S Ocean Blvd #503 in Highland Beach,"$800,000",2,2.0,"2,050",1982,$390,Beaches MLS,RX-11023545,Mary Ann Phillips RE/MAX Select Group,Mary Ann Phillips RE/MAX Select Group
7,2024-12-10,1748 Jupiter Cove Dr #220 in Jupiter,"$775,000",2,2.0,"1,300",1985,$596,Beaches MLS,RX-10984654,"Carol Hartmanis PA NV Realty Group, LLC",Jacqueline Hanna William Raveis Palm Beach LLC...
8,2024-12-11,108 Palm Bay Dr Unit A in Palm Beach Gardens,"$765,000",2,2.0,"2,195",2002,$349,Beaches MLS,RX-11016654,"Nancy Rossman Donohue Real Estate, LLC","Andrew Leibowitz Leibowitz Realty Group, LLC./..."
9,2024-12-09,100 Ocean Trail Way #1305 in Jupiter,"$745,000",2,2.0,"1,170",1982,$637,Beaches MLS,RX-11022203,Kellie Wilson Coldwell Banker Realty,Cibie Cahur Keller Williams Realty Jupiter
